# Batch Process Altinha Videos

This notebook processes multiple altinha videos using the reusable function from `altinha_processor.py`.

<a href="https://colab.research.google.com/github/kifjj/altinha-play/blob/main/batch_process_videos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Install Dependencies


In [ ]:
!pip install "numpy<2.0" "scipy<1.14" supervision ultralytics "opencv-python-headless<4.12"


## Import Processing Module

Upload `altinha_processor.py` to this environment, then import the processing function.


In [ ]:
# Import the processing function
# Make sure altinha_processor.py is uploaded to the working directory
from altinha_processor import process_altinha_video

# Alternatively, download from GitHub:
# !wget https://raw.githubusercontent.com/kifjj/altinha-play/main/altinha_processor.py
# from altinha_processor import process_altinha_video


## Import Additional Libraries


In [ ]:
import os
import json
from datetime import datetime
import pandas as pd


## Configuration

Set up model paths and output directories.


In [ ]:
# Model paths (shared across all videos)
BALL_MODEL_PATH = '/kaggle/input/yolo-ft-2511/pytorch/default/1/altinha_best.pt'
POSE_MODEL_PATH = '/kaggle/input/yolo11-pose/pytorch/default/1/yolo11l-pose.pt'

# Output directories
OUTPUT_DIR = '/kaggle/working/batch_results'
DEBUG_FRAMES_DIR = '/kaggle/working/debug_frames'

# Create output directories
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(DEBUG_FRAMES_DIR, exist_ok=True)

print(f"✅ Output directory: {OUTPUT_DIR}")
print(f"✅ Debug frames directory: {DEBUG_FRAMES_DIR}")


## Video List

Define the array of video paths to process. **Update this list with your video paths!**


In [ ]:
# List of video paths to process
# Update this array with your video file paths
VIDEO_PATHS = [
    '/kaggle/input/alta-videos/altinha-beach-green-mq-13s.mp4',
    '/kaggle/input/alta-videos/altinha-beach-dark-mq-20s.mp4',
    # Add more video paths here...
    # '/kaggle/input/alta-videos/video3.mp4',
    # '/kaggle/input/alta-videos/video4.mp4',
]

print(f"📹 Found {len(VIDEO_PATHS)} videos to process:")
for i, video_path in enumerate(VIDEO_PATHS, 1):
    print(f"   {i}. {os.path.basename(video_path)}")


## Process All Videos

Loop through all videos and process them one by one.


In [ ]:
# Store results for all videos
all_results = []

print("\n" + "#"*80)
print("#" + " "*78 + "#")
print(f"#  BATCH PROCESSING {len(VIDEO_PATHS)} VIDEOS".ljust(79) + "#")
print("#" + " "*78 + "#")
print("#"*80 + "\n")

start_time = datetime.now()

for idx, video_path in enumerate(VIDEO_PATHS, 1):
    print(f"\n{'='*80}")
    print(f"VIDEO {idx}/{len(VIDEO_PATHS)}: {os.path.basename(video_path)}")
    print(f"{'='*80}")
    
    try:
        # Generate output path
        video_basename = os.path.splitext(os.path.basename(video_path))[0]
        output_video_path = os.path.join(OUTPUT_DIR, f"{video_basename}_annotated.mp4")
        
        # Create video-specific debug directory
        video_debug_dir = os.path.join(DEBUG_FRAMES_DIR, video_basename)
        
        # Process the video
        result = process_altinha_video(
            video_path=video_path,
            model_path=BALL_MODEL_PATH,
            output_path=output_video_path,
            pose_model_path=POSE_MODEL_PATH,
            debug_frames_dir=video_debug_dir,
            verbose=True  # Set to False to reduce output
        )
        
        # Store result
        result['status'] = 'success'
        result['error'] = None
        all_results.append(result)
        
        print(f"\n✅ Successfully processed: {os.path.basename(video_path)}")
        
    except Exception as e:
        print(f"\n❌ Error processing {os.path.basename(video_path)}: {str(e)}")
        all_results.append({
            'video_name': os.path.basename(video_path),
            'status': 'error',
            'error': str(e),
            'output_path': None,
            'ball_detection_percentage': 0,
            'total_hits': 0,
            'head_hits': 0,
            'foot_hits': 0,
            'unknown_hits': 0,
        })

end_time = datetime.now()
total_duration = (end_time - start_time).total_seconds()

print(f"\n\n{'#'*80}")
print(f"#  BATCH PROCESSING COMPLETE".ljust(79) + "#")
print(f"#  Total time: {total_duration:.1f} seconds".ljust(79) + "#")
print("#"*80)


In [ ]:
print("\n" + "="*80)
print("BATCH PROCESSING SUMMARY")
print("="*80 + "\n")

# Create summary DataFrame
summary_data = []
for result in all_results:
    summary_data.append({
        'Video': result['video_name'],
        'Status': result['status'],
        'Ball Detection %': f"{result['ball_detection_percentage']:.1%}" if result['status'] == 'success' else 'N/A',
        'Total Hits': result['total_hits'],
        'Head Hits': result['head_hits'],
        'Foot Hits': result['foot_hits'],
        'Unknown Hits': result['unknown_hits'],
    })

df_summary = pd.DataFrame(summary_data)
print(df_summary.to_string(index=False))

# Overall statistics
successful = sum(1 for r in all_results if r['status'] == 'success')
failed = len(all_results) - successful
total_hits = sum(r['total_hits'] for r in all_results)
avg_detection = sum(r['ball_detection_percentage'] for r in all_results if r['status'] == 'success') / successful if successful > 0 else 0

print(f"\n{'='*80}")
print(f"✅ Successful: {successful}/{len(all_results)} videos")
print(f"❌ Failed: {failed}/{len(all_results)} videos")
print(f"⚽ Total hits across all videos: {total_hits}")
print(f"📊 Average ball detection rate: {avg_detection:.1%}")
print("="*80)


## Save Batch Summary

Save comprehensive JSON and CSV summaries of all processed videos.


In [ ]:
# Create batch summary
batch_summary = {
    'batch_info': {
        'timestamp': datetime.now().isoformat(),
        'total_videos': len(all_results),
        'successful': successful,
        'failed': failed,
        'total_processing_time_sec': total_duration,
    },
    'aggregate_stats': {
        'total_hits': total_hits,
        'average_detection_percentage': round(avg_detection, 4),
    },
    'videos': []
}

# Add individual video results
for result in all_results:
    video_summary = {
        'video_name': result['video_name'],
        'status': result['status'],
        'output_path': result.get('output_path'),
        'json_path': result.get('json_path'),
    }
    
    if result['status'] == 'success':
        video_summary.update({
            'ball_detection_percentage': round(result['ball_detection_percentage'], 4),
            'total_hits': result['total_hits'],
            'head_hits': result['head_hits'],
            'foot_hits': result['foot_hits'],
            'unknown_hits': result['unknown_hits'],
            'hit_frames': [h['frame'] for h in result.get('hit_detections', [])],
        })
    else:
        video_summary['error'] = result.get('error')
    
    batch_summary['videos'].append(video_summary)

# Save batch summary JSON
batch_summary_path = os.path.join(OUTPUT_DIR, 'batch_summary.json')
with open(batch_summary_path, 'w') as f:
    json.dump(batch_summary, f, indent=2)

print(f"\n📊 Batch summary saved to: {batch_summary_path}")

# Save as CSV for easy viewing
csv_path = os.path.join(OUTPUT_DIR, 'batch_summary.csv')
df_summary.to_csv(csv_path, index=False)
print(f"📊 CSV summary saved to: {csv_path}")


In [ ]:
# Optional: Create a zip file of all results for easy download
import shutil

zip_path = '/kaggle/working/batch_results'
shutil.make_archive(zip_path, 'zip', OUTPUT_DIR)

print(f"\n📦 All results zipped to: {zip_path}.zip")
print(f"   You can download this file from the output section.")
